## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `dataset-prep.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from datasets import load_dataset

In [3]:
datasets = load_dataset("hate-speech-portuguese/hate_speech_portuguese", split='train[:10%]')

In [4]:
print(datasets)

Dataset({
    features: ['text', 'label', 'hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3'],
    num_rows: 567
})


In [5]:
datasets = datasets.remove_columns(['hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3'])
datasets

Dataset({
    features: ['text', 'label'],
    num_rows: 567
})

In [6]:
datasets = datasets.train_test_split(test_size=0.2)

In [7]:
datasets['train']['text'][0]

'A mídia DE NOTÍCIAS FALSAS desconhece a verdade deliberadamente. Grande perigo ao país. O @nytimes falido virou pia _ https://t.co/ssibMARmkc'

In [8]:
def removeN(example):
    example["text"] = example["text"].replace("\n", " ")
    return example

In [9]:
datasets = datasets.map(removeN)

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [10]:
datasets['train']['text'][0]

'A mídia DE NOTÍCIAS FALSAS desconhece a verdade deliberadamente. Grande perigo ao país. O @nytimes falido virou pia _ https://t.co/ssibMARmkc'

### label 0 -> No Hate Speech
### label 1 -> Hate Speech

In [11]:
def labelChange(example):
    example["label_text"] = "No Hate Speech" if  example["label"] == 0 else "Hate Speech"
    return example

In [12]:
datasets = datasets.map(labelChange)
datasets

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 453
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 114
    })
})

In [13]:
datasets = datasets.remove_columns(['label'])
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label_text'],
        num_rows: 453
    })
    test: Dataset({
        features: ['text', 'label_text'],
        num_rows: 114
    })
})

In [14]:
datasets['train'][0]

{'text': 'A mídia DE NOTÍCIAS FALSAS desconhece a verdade deliberadamente. Grande perigo ao país. O @nytimes falido virou pia _ https://t.co/ssibMARmkc',
 'label_text': 'No Hate Speech'}

In [15]:
# CONSTRUÇÃO DO OBJETO PARA OPEN AI
def dataset_to_json(datasets, file_name):
    with open(file_name, 'w', encoding="utf-8") as f:
        for example in datasets:
            json_obj = {
                "messages": [
                    {"role": "system", "content": "Seu trabalho é classificar os comentários do usuário em Hate Speech ou No Hate Speech."},
                    {"role": "user", "content": example["text"]},
                    {"role": "assistant", "content": example["label_text"]}
                ]
            }
            f.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

In [16]:
dataset_to_json(datasets["train"], "train.jsonl")

In [17]:
dataset_to_json(datasets["test"], "validation.jsonl")

In [18]:
from openai import OpenAI
client = OpenAI()

In [22]:
file_train = client.files.create(
    file=open("train.jsonl", "rb"),
    purpose="fine-tune",
)

In [23]:
file_validation = client.files.create(
    file=open("validation.jsonl", "rb"),
    purpose="fine-tune",
)

In [21]:
# Devido ao Fine-tune ter sido descontinuado para modelo davinci-002 estarei utilizando o modelo mais recente até momento de saída da aula e também recomendado pela documentação
# https://platform.openai.com/docs/guides/fine-tuning#create-a-fine-tuned-model
client.fine_tuning.jobs.create(
    training_file=file_train.id,
    validation_file=file_validation.id,
    model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-WJ260o5JFtFYO2Lc56LC6Xi0', created_at=1737333203, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-9Ka4MdZAwINdpOMmxZxue32b', result_files=[], seed=568076911, status='validating_files', trained_tokens=None, training_file='file-Pq8eAq7LNP69yb8nPerGxF', validation_file='file-UMJvtV1xTSft5WkTbkND1N', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)